# Рабочий процесс с участием человека в Microsoft Agent Framework

## 🎯 Цели обучения

В этом ноутбуке вы узнаете, как реализовать рабочие процессы с участием человека, используя `RequestInfoExecutor` из Microsoft Agent Framework. Этот мощный подход позволяет приостанавливать выполнение AI-агентов для получения ввода от человека, делая их интерактивными и предоставляя людям контроль над важными решениями.

## 🔄 Что такое участие человека?

**Участие человека (Human-in-the-loop, HITL)** — это подход, при котором AI-агенты приостанавливают выполнение, чтобы запросить ввод от человека перед продолжением. Это важно для:

- ✅ **Критических решений** - Получение одобрения человека перед выполнением важных действий
- ✅ **Неоднозначных ситуаций** - Позволить человеку уточнить, если AI не уверен
- ✅ **Предпочтений пользователя** - Запросить выбор пользователя между несколькими вариантами
- ✅ **Соответствия и безопасности** - Обеспечить контроль человека для регулируемых операций
- ✅ **Интерактивных взаимодействий** - Создание разговорных агентов, которые реагируют на ввод пользователя

## 🏗️ Как это работает в Microsoft Agent Framework

Фреймворк предоставляет три ключевых компонента для HITL:

1. **`RequestInfoExecutor`** - Специальный исполнитель, который приостанавливает рабочий процесс и генерирует событие `RequestInfoEvent`
2. **`RequestInfoMessage`** - Базовый класс для типизированных запросов, отправляемых человеку
3. **`RequestResponse`** - Соотносит ответы человека с исходными запросами, используя `request_id`

**Шаблон рабочего процесса:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Наш пример: Бронирование отеля с подтверждением пользователя

Мы расширим условный рабочий процесс, добавив подтверждение от человека **перед** предложением альтернативных направлений:

1. Пользователь запрашивает направление (например, "Париж")
2. `availability_agent` проверяет доступность номеров
3. **Если номеров нет** → `confirmation_agent` спрашивает: "Хотите ли вы увидеть альтернативы?"
4. Рабочий процесс **приостанавливается** с помощью `RequestInfoExecutor`
5. **Человек отвечает** "да" или "нет" через ввод в консоли
6. `decision_manager` направляет в зависимости от ответа:
   - **Да** → Показать альтернативные направления
   - **Нет** → Отменить запрос на бронирование
7. Отображение окончательного результата

Это демонстрирует, как предоставить пользователям контроль над предложениями агента!

---

Давайте начнем! 🚀


## Шаг 1: Импорт необходимых библиотек

Мы импортируем стандартные компоненты Agent Framework, а также **классы, специфичные для взаимодействия с человеком**:
- `RequestInfoExecutor` - Исполнитель, который приостанавливает выполнение рабочего процесса для ввода данных человеком
- `RequestInfoEvent` - Событие, которое генерируется при запросе ввода данных человеком
- `RequestInfoMessage` - Базовый класс для типизированных запросов
- `RequestResponse` - Связывает ответы человека с запросами
- `WorkflowOutputEvent` - Событие для обнаружения выходных данных рабочего процесса


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Шаг 2: Определение моделей Pydantic для структурированных выводов

Эти модели определяют **схему**, которую будут возвращать агенты. Мы сохраняем все модели из условного рабочего процесса и добавляем:

**Новое для участия человека в процессе:**
- `HumanFeedbackRequest` - Подкласс `RequestInfoMessage`, который определяет данные запроса, отправляемые людям
  - Содержит `prompt` (вопрос для задавания) и `destination` (контекст о недоступном городе)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Шаг 3: Создание инструмента бронирования отеля

Тот же инструмент из условного рабочего процесса - проверяет доступность номеров в пункте назначения.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Шаг 4: Определение функций условий для маршрутизации

Нам нужны **четыре функции условий** для нашего рабочего процесса с участием человека:

**Из условного рабочего процесса:**
1. `has_availability_condition` - Направляет, когда отели ДОСТУПНЫ
2. `no_availability_condition` - Направляет, когда отели НЕ ДОСТУПНЫ

**Новые для рабочего процесса с участием человека:**
3. `user_wants_alternatives_condition` - Направляет, когда пользователь отвечает "да" на альтернативы
4. `user_declines_alternatives_condition` - Направляет, когда пользователь отвечает "нет" на альтернативы


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Шаг 5: Создание исполнителя Decision Manager

Это **основа паттерна "человек в цикле"!** `DecisionManager` — это пользовательский `Executor`, который:

1. **Получает обратную связь от человека** через объекты `RequestResponse`
2. **Обрабатывает решение пользователя** (да/нет)
3. **Направляет рабочий процесс**, отправляя сообщения соответствующим агентам

Ключевые особенности:
- Использует декоратор `@handler` для предоставления методов в качестве шагов рабочего процесса
- Получает `RequestResponse[HumanFeedbackRequest, str]`, содержащий как исходный запрос, так и ответ пользователя
- Генерирует простые сообщения "да" или "нет", которые активируют наши функции условий


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Шаг 6: Создание пользовательского исполнителя отображения

Тот же исполнитель отображения из условного рабочего процесса - выдает окончательные результаты в качестве вывода рабочего процесса.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Шаг 7: Загрузка переменных окружения

Настройте клиент LLM (GitHub Models, Azure OpenAI или OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Шаг 8: Создание AI-агентов и исполнителей

Мы создаем **шесть компонентов рабочего процесса**:

**Агенты (обернутые в AgentExecutor):**
1. **availability_agent** - Проверяет доступность отелей с помощью инструмента
2. **confirmation_agent** - 🆕 Готовит запрос на подтверждение от человека
3. **alternative_agent** - Предлагает альтернативные города (когда пользователь соглашается)
4. **booking_agent** - Побуждает к бронированию (когда номера доступны)
5. **cancellation_agent** - 🆕 Обрабатывает сообщение об отмене (когда пользователь отказывается)

**Специальные исполнители:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor`, который приостанавливает рабочий процесс для ввода данных человеком
7. **decision_manager** - 🆕 Пользовательский исполнитель, который направляет процесс на основе ответа человека (уже определен выше)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Шаг 9: Построение рабочего процесса с участием человека

Теперь мы создаем граф рабочего процесса с **условной маршрутизацией**, включая путь с участием человека:

**Структура рабочего процесса:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Ключевые связи:**
- `availability_agent → confirmation_agent` (когда нет свободных номеров)
- `confirmation_agent → prepare_human_request` (изменение типа)
- `prepare_human_request → request_info_executor` (пауза для участия человека)
- `request_info_executor → decision_manager` (всегда - предоставляет RequestResponse)
- `decision_manager → alternative_agent` (когда пользователь говорит "да")
- `decision_manager → cancellation_agent` (когда пользователь говорит "нет")
- `availability_agent → booking_agent` (когда номера доступны)
- Все пути заканчиваются на `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Шаг 10: Запуск тестового случая 1 - Город БЕЗ доступности (Париж с подтверждением человеком)

Этот тест демонстрирует **полный цикл с участием человека**:

1. Запрос на бронирование отеля в Париже
2. availability_agent проверяет → Нет свободных номеров
3. confirmation_agent создает вопрос для человека
4. request_info_executor **приостанавливает выполнение** и генерирует `RequestInfoEvent`
5. **Приложение обнаруживает событие и запрашивает пользователя через консоль**
6. Пользователь вводит "да" или "нет"
7. Приложение отправляет ответ через `send_responses_streaming()`
8. decision_manager направляет в зависимости от ответа
9. Отображается окончательный результат

**Основной шаблон:**
- Используйте `workflow.run_stream()` для первого цикла
- Используйте `workflow.send_responses_streaming(pending_responses)` для последующих циклов
- Слушайте `RequestInfoEvent`, чтобы определить, когда требуется ввод от человека
- Слушайте `WorkflowOutputEvent`, чтобы получить окончательные результаты


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Шаг 11: Запуск тестового случая 2 - Город С Наличием (Стокгольм - Ввод человека не требуется)

Этот тест демонстрирует **прямой путь**, когда номера доступны:

1. Запросить отель в Стокгольме
2. availability_agent проверяет → Номера доступны ✅
3. booking_agent предлагает бронирование
4. display_result показывает подтверждение
5. **Ввод человека не требуется!**

Рабочий процесс полностью обходит путь с участием человека, если номера доступны.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Основные выводы и лучшие практики для взаимодействия человека и системы

### ✅ Что вы узнали:

#### 1. **Шаблон RequestInfoExecutor**
Шаблон взаимодействия человека и системы в Microsoft Agent Framework использует три ключевых компонента:
- `RequestInfoExecutor` - Приостанавливает выполнение и генерирует события
- `RequestInfoMessage` - Базовый класс для типизированных запросов (наследуйте его!)
- `RequestResponse` - Соотносит ответы человека с исходными запросами

**Ключевые моменты:**
- `RequestInfoExecutor` НЕ собирает данные самостоятельно - он только приостанавливает выполнение
- Ваше приложение должно слушать `RequestInfoEvent` и собирать данные
- Вы должны вызвать `send_responses_streaming()` с словарем, сопоставляющим `request_id` с ответом пользователя

#### 2. **Шаблон потокового выполнения**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Событийно-ориентированная архитектура**
Слушайте определенные события для управления выполнением:
- `RequestInfoEvent` - Требуется ввод человека (выполнение приостановлено)
- `WorkflowOutputEvent` - Доступен окончательный результат (выполнение завершено)
- `WorkflowStatusEvent` - Изменения состояния (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS и т.д.)

#### 4. **Пользовательские исполнители с @handler**
`DecisionManager` демонстрирует, как создавать исполнителей, которые:
- Используют декоратор `@handler` для предоставления методов в качестве шагов выполнения
- Получают типизированные сообщения (например, `RequestResponse[HumanFeedbackRequest, str]`)
- Направляют выполнение, отправляя сообщения другим исполнителям
- Получают доступ к контексту через `WorkflowContext`

#### 5. **Условная маршрутизация с решениями человека**
Вы можете создавать функции условий, которые оценивают ответы человека:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Применение в реальном мире:

1. **Процессы утверждения**
   - Получение одобрения менеджера перед обработкой отчетов о расходах
   - Требование проверки человеком перед отправкой автоматизированных писем
   - Подтверждение транзакций с высокой стоимостью перед выполнением

2. **Модерация контента**
   - Пометка сомнительного контента для проверки человеком
   - Запрос модераторов для принятия окончательного решения в сложных случаях
   - Передача на рассмотрение человеку, если уверенность ИИ низкая

3. **Обслуживание клиентов**
   - Позволить ИИ автоматически обрабатывать рутинные вопросы
   - Передача сложных вопросов человеческим агентам
   - Запрос клиента, хочет ли он поговорить с человеком

4. **Обработка данных**
   - Запрос человека для разрешения неоднозначных записей данных
   - Подтверждение интерпретации ИИ неясных документов
   - Позволить пользователям выбирать между несколькими допустимыми интерпретациями

5. **Системы критической безопасности**
   - Требование подтверждения человеком перед необратимыми действиями
   - Получение одобрения перед доступом к конфиденциальным данным
   - Подтверждение решений в регулируемых отраслях (здравоохранение, финансы)

6. **Интерактивные агенты**
   - Создание разговорных ботов, которые задают уточняющие вопросы
   - Разработка мастеров, которые проводят пользователей через сложные процессы
   - Проектирование агентов, которые сотрудничают с людьми шаг за шагом

### 🔄 Сравнение: с человеком в цикле и без него

| Функция | Условный рабочий процесс | Рабочий процесс с человеком в цикле |
|---------|--------------------------|-------------------------------------|
| **Выполнение** | Один `workflow.run()` | Цикл с `run_stream()` + `send_responses_streaming()` |
| **Ввод пользователя** | Нет (полностью автоматизировано) | Интерактивные запросы через `input()` или UI |
| **Компоненты** | Агенты + Исполнители | + RequestInfoExecutor + DecisionManager |
| **События** | Только AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent и др. |
| **Приостановка** | Без приостановки | Выполнение приостанавливается на RequestInfoExecutor |
| **Контроль человека** | Без контроля человека | Люди принимают ключевые решения |
| **Сценарий использования** | Автоматизация | Сотрудничество и контроль |

### 🚀 Продвинутые шаблоны:

#### Несколько точек принятия решений человеком
Вы можете иметь несколько узлов `RequestInfoExecutor` в одном рабочем процессе:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Обработка тайм-аутов
Реализуйте тайм-ауты для ответов человека:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Интеграция с богатым интерфейсом
Вместо `input()` интегрируйте с веб-интерфейсом, Slack, Teams и др.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Условное взаимодействие человека и системы
Запрашивайте ввод человека только в определенных ситуациях:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Лучшие практики:

1. **Всегда наследуйте RequestInfoMessage**
   - Обеспечивает безопасность типов и валидацию
   - Позволяет предоставлять богатый контекст для отображения в интерфейсе
   - Уточняет намерение каждого типа запроса

2. **Используйте описательные запросы**
   - Включайте контекст того, что вы спрашиваете
   - Объясняйте последствия каждого выбора
   - Делайте вопросы простыми и понятными

3. **Обрабатывайте неожиданный ввод**
   - Проверяйте ответы пользователя
   - Предоставляйте значения по умолчанию для некорректного ввода
   - Давайте четкие сообщения об ошибках

4. **Отслеживайте идентификаторы запросов**
   - Используйте связь между request_id и ответами
   - Не пытайтесь управлять состоянием вручную

5. **Проектируйте для асинхронности**
   - Не блокируйте потоки, ожидая ввода
   - Используйте асинхронные шаблоны
   - Поддерживайте одновременные экземпляры рабочих процессов

### 📚 Связанные концепции:

- **Промежуточное ПО агента** - Перехват вызовов агента (предыдущая тетрадь)
- **Управление состоянием рабочего процесса** - Сохранение состояния рабочего процесса между запусками
- **Сотрудничество нескольких агентов** - Совмещение взаимодействия человека и системы с командами агентов
- **Событийно-ориентированные архитектуры** - Создание реактивных систем с использованием событий

---

### 🎓 Поздравляем!

Вы освоили рабочие процессы взаимодействия человека и системы с Microsoft Agent Framework! Теперь вы знаете, как:
- ✅ Приостанавливать рабочие процессы для сбора ввода человека
- ✅ Использовать RequestInfoExecutor и RequestInfoMessage
- ✅ Обрабатывать потоковое выполнение с помощью событий
- ✅ Создавать пользовательские исполнители с @handler
- ✅ Направлять рабочие процессы на основе решений человека
- ✅ Создавать интерактивных ИИ-агентов, которые сотрудничают с людьми

**Это важный шаблон для создания надежных и управляемых ИИ-систем!** 🚀



---

**Отказ от ответственности**:  
Этот документ был переведен с использованием сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
